# 🚀 Introduction to AI Agent Deployment

## 🛒 The Zava Scenario

Cora has been evaluated, tested, and is ready for customers. But deploying an AI agent to production is different from deploying traditional applications. How do we ensure Cora can handle production traffic? What about managing costs and updates?

**The Reality**: Production deployment requires careful planning around capacity management, monitoring, rollout strategies, and operational readiness.

## What You'll Learn

In this section, you'll understand:

1. **Model deployment fundamentals** - From development to production
2. **Capacity planning** - PTUs vs. pay-per-use pricing models
3. **Deployment strategies** - Blue-green and canary deployments
4. **Monitoring and observability** - Tracking performance in production
5. **Production readiness checklist** - Key considerations before launch

## Why This Matters

Successful deployment ensures:
- **Reliability** - Cora is available when customers need it
- **Performance** - Fast response times even under load
- **Cost optimization** - Right-sized infrastructure for your needs
- **Safe updates** - Rolling out improvements without disruption
- **Operational excellence** - Monitoring and incident response capabilities

Let's explore how to deploy Cora to production with confidence.

---

## What is Model Deployment?

**Model deployment** is the process of making your AI model available for production use.

### Development vs. Production

**Development environment:**
- Test models locally or in notebooks
- Experiment with different configurations
- Limited users (just you and your team)
- Failures are acceptable (learning process)
- Can restart/change anytime

**Production environment:**
- Model serves real customers
- Must be reliable and performant
- High availability required
- Failures impact business
- Changes must be carefully managed

**Deployment bridges this gap.**

### What Gets Deployed

**For base models:**
- Model already deployed by Azure
- You just reference it (e.g., "gpt-4o-mini")
- No deployment action needed

**For fine-tuned models:**
- Custom model created by your fine-tuning job
- Exists in Azure but not yet accessible
- Must be **deployed** to make it available via API
- Gets a deployment name you can reference

**Analogy:**
```
Fine-tuning = Building a custom car
Deployment = Putting it on the road for use
```

### The Deployment Process

```
1. Fine-Tuning Job Completes
   ↓
   Model created (fine-tuned-model-xyz)
   
2. Create Deployment
   ↓
   Deploy model to inference endpoint
   Assign capacity (tokens per minute)
   Name deployment (e.g., "cora-v1")
   
3. Deployment Ready
   ↓
   Model accessible via API
   Can process customer requests
   
4. Start Using
   ↓
   Update application to use new deployment
   Route traffic to deployed model
```

---

## Why Deploy Fine-Tuned Models?

### The Problem Without Deployment

**After fine-tuning:**
```python
# Your fine-tuning job completes
job_status = "succeeded"
model_id = "ft:gpt-4o-mini:zava:abc123"

# Try to use it
response = client.chat.completions.create(
    model=model_id,  # This won't work yet!
    messages=[...]
)
# → Error: Model not found or not deployed
```

**The fine-tuned model exists but isn't deployed.**

### After Deployment

```python
# Deploy the model first
deployment_name = "cora-fine-tuned-v1"

# Now you can use it
response = client.chat.completions.create(
    model=deployment_name,  # ✓ Works!
    messages=[...]
)
```

**Deployment makes the model accessible.**

### Benefits of Proper Deployment

**1. API Access**
- Model available via standard API
- Consistent interface with other models
- Easy to integrate into applications

**2. Capacity Management**
- Control tokens per minute (TPM)
- Scale up/down as needed
- Prevent overload

**3. Version Control**
- Multiple deployments for same model
- A/B testing (v1 vs. v2)
- Rollback capability

**4. Security & Access**
- Managed identity integration
- Role-based access control
- Network isolation options

**5. Monitoring**
- Track usage and performance
- Monitor costs
- Detect issues early

---

## Deployment Concepts

### Deployment Name

**What it is:** A friendly name for your deployed model

**Purpose:**
- Reference model in API calls
- Distinguish between versions
- Organize multiple deployments

**Examples:**
```python
# Good deployment names
"cora-fine-tuned-v1"
"zava-customer-service-prod"
"product-assistant-2024-11"

# Less clear
"my-model"
"test"
"deployment1"
```

**Best practice:** Use descriptive, versioned names

### Model ID vs. Deployment Name

**Model ID:**
- Created by fine-tuning job
- Immutable (doesn't change)
- Example: `ft:gpt-4o-mini:zava:abc123::xyz789`

**Deployment Name:**
- Chosen by you when deploying
- Mutable (can redeploy)
- Example: `cora-v1`

```python
# Deploy model (one-time)
create_deployment(
    model=model_id,              # What to deploy
    name=deployment_name         # What to call it
)

# Use model (many times)
query_model(
    model=deployment_name,       # Use friendly name
    prompt=customer_question
)
```

### Capacity and Scaling

**Capacity = Tokens Per Minute (TPM)**

**What it means:**
- Maximum tokens your deployment can process per minute
- Includes input + output tokens
- Determines throughput

**Example:**
```
Deployment capacity: 10,000 TPM

Customer query: 100 tokens
Typical response: 150 tokens
Total per request: 250 tokens

Max queries/minute: 10,000 / 250 = 40 queries
Max queries/day: 40 × 60 × 24 = 57,600 queries
```

**Choosing capacity:**

| Use Case | Recommended TPM | Reasoning |
|----------|-----------------|-----------|
| **Development/Testing** | 1,000 - 10,000 | Low volume, cost-effective |
| **Small Production** | 10,000 - 50,000 | Hundreds of users |
| **Medium Production** | 50,000 - 200,000 | Thousands of users |
| **Large Production** | 200,000+ | High volume, enterprise scale |

**Scaling strategies:**

```python
# Start small
initial_capacity = 10_000  # TPM

# Monitor usage
actual_usage = get_usage_metrics()

# Scale up if needed
if actual_usage > 0.8 * initial_capacity:
    new_capacity = initial_capacity * 2
    update_deployment(capacity=new_capacity)
```

### Deployment Region

**Considerations:**

**1. Latency**
- Deploy close to users
- Reduces response time
- Better user experience

**2. Model Availability**
- Not all models in all regions
- Fine-tuning available in specific regions
- Check availability before deploying

**3. Compliance**
- Data residency requirements
- Regional regulations
- Company policies

**Example regions:**
- East US
- West Europe
- Sweden Central (popular for fine-tuning)
- North Central US

**Best practice:** Deploy in same region as your fine-tuning job

---

## Deployment Process Step-by-Step

### Step 1: Verify Fine-Tuning Job

**Before deploying, ensure fine-tuning succeeded:**

```python
from openai import AzureOpenAI

client = AzureOpenAI(...)

# Get fine-tuning job
job = client.fine_tuning.jobs.retrieve(job_id)

# Check status
print(f"Status: {job.status}")
print(f"Model ID: {job.fine_tuned_model}")

# Verify success
if job.status == "succeeded":
    print("✓ Fine-tuning complete, ready to deploy")
    model_id = job.fine_tuned_model
else:
    print(f"✗ Job not ready: {job.status}")
```

**Possible statuses:**
- `succeeded` → Ready to deploy ✓
- `running` → Wait for completion
- `failed` → Check errors, retry
- `cancelled` → Start new job

### Step 2: Create Deployment

**Using Azure OpenAI SDK:**

```python
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Connect to Azure
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=AZURE_SUBSCRIPTION_ID,
    resource_group_name=AZURE_RESOURCE_GROUP,
    workspace_name=AZURE_AI_PROJECT_NAME
)

# Create deployment
deployment = ml_client.online_deployments.begin_create_or_update(
    deployment_name="cora-fine-tuned-v1",
    model=model_id,
    sku={
        "name": "Standard",
        "capacity": 10  # Tokens per minute (in thousands)
    }
)

# Wait for completion
deployment.wait()
print("✓ Deployment ready")
```

**Or using Azure Portal:**

1. Go to Azure AI Foundry (https://ai.azure.com)
2. Select your project
3. Click "Deployments" → "Create deployment"
4. Choose your fine-tuned model
5. Set deployment name and capacity
6. Click "Deploy"

### Step 3: Verify Deployment

**Test the deployed model:**

```python
# Query deployed model
response = client.chat.completions.create(
    model="cora-fine-tuned-v1",  # Your deployment name
    messages=[
        {"role": "system", "content": "You are Cora, a Zava assistant"},
        {"role": "user", "content": "What paint do you have?"}
    ]
)

print(response.choices[0].message.content)
# → Should respond in Cora's fine-tuned style
```

**Verification checklist:**
- ✓ Model responds without errors
- ✓ Response quality matches expectations
- ✓ Latency is acceptable (< 2-3 seconds)
- ✓ Style matches fine-tuned behavior

### Step 4: Update Application

**Point your application to the new deployment:**

```python
# Before (using base model)
MODEL_NAME = "gpt-4o-mini"

# After (using fine-tuned deployment)
MODEL_NAME = "cora-fine-tuned-v1"

# Application code stays the same
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[...]
)
```

**For gradual rollout:**

```python
import random

def get_model_name():
    """Route 10% to new model, 90% to old"""
    if random.random() < 0.1:
        return "cora-fine-tuned-v1"  # New
    else:
        return "gpt-4o-mini"  # Old (base)

# Use in application
response = client.chat.completions.create(
    model=get_model_name(),
    messages=[...]
)
```

---

## Model Management Best Practices

### 1. Version Your Deployments

**Use semantic versioning:**

```python
# Version naming
"cora-v1.0"  # Initial release
"cora-v1.1"  # Minor update (bug fix)
"cora-v2.0"  # Major update (new capabilities)

# Date-based versioning
"cora-2024-11-13"
"cora-november-prod"

# Environment-based
"cora-dev"
"cora-staging"
"cora-prod"
```

**Benefits:**
- Clear version history
- Easy rollback if needed
- A/B testing between versions

### 2. Maintain Multiple Deployments

```python
deployments = {
    "cora-prod": {
        "model": "ft:gpt-4o-mini:zava:v1",
        "capacity": 50_000,
        "purpose": "Production traffic"
    },
    "cora-staging": {
        "model": "ft:gpt-4o-mini:zava:v2",
        "capacity": 10_000,
        "purpose": "Testing new version"
    },
    "cora-dev": {
        "model": "ft:gpt-4o-mini:zava:v3",
        "capacity": 1_000,
        "purpose": "Development experiments"
    }
}
```

**Workflow:**
```
1. Develop → test in cora-dev
2. Validate → promote to cora-staging
3. Production test → small % to cora-staging
4. Full rollout → switch cora-prod
```

### 3. Implement Rollback Strategy

**Quick rollback if issues occur:**

```python
# Configuration
deployments = {
    "current": "cora-v2.0",
    "previous": "cora-v1.0"
}

# If v2.0 has issues
def rollback():
    """Switch back to previous version"""
    deployments["current"] = deployments["previous"]
    print(f"Rolled back to {deployments['current']}")

# Monitor for issues
if error_rate > threshold:
    rollback()
```

**Keep previous versions deployed for quick switching.**

### 4. Monitor Deployment Health

**Key metrics to track:**

```python
# Usage metrics
total_requests = count_api_calls()
tokens_used = sum_token_usage()
error_rate = count_errors() / total_requests

# Performance metrics
avg_latency = mean_response_time()
p95_latency = percentile_95_response_time()

# Cost metrics
daily_cost = tokens_used * price_per_token

# Quality metrics
satisfaction_score = average_user_rating()
```

**Set up alerts:**

```python
alerts = {
    "error_rate": {"threshold": 0.05, "action": "page_on_call"},
    "latency": {"threshold": 3000, "action": "investigate"},
    "cost": {"threshold": 1000, "action": "notify_team"}
}
```

### 5. Document Deployments

**Maintain deployment registry:**

```yaml
# deployments.yaml
deployments:
  cora-v1.0:
    model_id: ft:gpt-4o-mini:zava:abc123
    deployed: 2024-11-01
    status: production
    capacity: 50000
    notes: Initial fine-tuned model
    
  cora-v2.0:
    model_id: ft:gpt-4o-mini:zava:xyz789
    deployed: 2024-11-13
    status: staging
    capacity: 10000
    notes: Improved product knowledge
```

**Track changes over time.**

---

## Production Readiness Checklist

Before deploying to production, verify:

### ✅ Quality Validation

- [ ] Evaluation metrics meet thresholds (groundedness > 4.0, etc.)
- [ ] Safety checks pass (no high-severity content violations)
- [ ] Tested on diverse query types
- [ ] Edge cases handled gracefully
- [ ] Regression tests pass (didn't break existing functionality)

### ✅ Performance Testing

- [ ] Latency < 3 seconds for 95th percentile
- [ ] Load tested at expected traffic volume
- [ ] Capacity sufficient for peak usage
- [ ] No memory leaks or resource exhaustion
- [ ] Graceful degradation under high load

### ✅ Security & Compliance

- [ ] Managed identity configured
- [ ] Role-based access control (RBAC) set up
- [ ] Content filtering enabled
- [ ] No sensitive data in prompts/responses
- [ ] Compliance requirements met (GDPR, etc.)

### ✅ Monitoring & Observability

- [ ] Application Insights configured
- [ ] Logging enabled (errors, warnings)
- [ ] Tracing set up (OpenTelemetry)
- [ ] Alerts configured (errors, latency, cost)
- [ ] Dashboard created for key metrics

### ✅ Operational Readiness

- [ ] Deployment runbook documented
- [ ] Rollback procedure tested
- [ ] On-call rotation established
- [ ] Incident response plan ready
- [ ] Capacity planning done (6-12 months)

### ✅ Business Validation

- [ ] Stakeholder approval obtained
- [ ] Budget approved for production costs
- [ ] User acceptance testing complete
- [ ] Support team trained
- [ ] Communication plan ready (users, customers)

### Example Checklist in Code

```python
def is_production_ready(deployment):
    """Validate deployment meets production standards"""
    checks = {
        "evaluation_passed": check_evaluation_metrics(deployment),
        "performance_ok": check_latency(deployment) < 3000,
        "security_configured": check_rbac(deployment),
        "monitoring_enabled": check_app_insights(deployment),
        "capacity_sufficient": check_capacity(deployment) > expected_load,
        "rollback_tested": check_rollback_works(deployment)
    }
    
    all_passed = all(checks.values())
    
    if not all_passed:
        failed = [k for k, v in checks.items() if not v]
        print(f"Failed checks: {failed}")
        
    return all_passed

if is_production_ready("cora-v2.0"):
    deploy_to_production("cora-v2.0")
else:
    print("Not ready for production yet")
```

---

## Deployment Strategies

### 1. Blue-Green Deployment

**Maintain two identical environments:**

```
Blue (current production):
- Deployment: cora-blue
- Handles 100% traffic
- Known stable version

Green (new version):
- Deployment: cora-green  
- Handles 0% traffic initially
- New version being validated

Cutover:
1. Deploy new version to Green
2. Test Green thoroughly
3. Switch traffic: Blue → Green
4. Keep Blue as rollback option
```

**Benefits:**
- Zero-downtime deployment
- Instant rollback (switch back to Blue)
- Full testing before production traffic

**Implementation:**

```python
# Traffic router
def route_traffic(use_green=False):
    if use_green:
        return "cora-green"
    else:
        return "cora-blue"

# Gradual cutover
MODEL = route_traffic(use_green=True)
```

### 2. Canary Deployment

**Gradually roll out to small % of users:**

```
Phase 1: 5% traffic to new version
  → Monitor for 24 hours
  → Check metrics
  
Phase 2: 25% traffic to new version
  → Monitor for 24 hours
  → Check metrics
  
Phase 3: 50% traffic to new version
  → Monitor for 24 hours
  → Check metrics
  
Phase 4: 100% traffic to new version
  → Full rollout complete
```

**Implementation:**

```python
import random

def get_deployment():
    """Route traffic based on rollout percentage"""
    rollout_percentage = 0.25  # 25% to new version
    
    if random.random() < rollout_percentage:
        return "cora-v2.0"  # New
    else:
        return "cora-v1.0"  # Old

MODEL = get_deployment()
```

**Benefits:**
- Limit blast radius of issues
- Real-world validation
- Easy rollback (reduce percentage)

### 3. A/B Testing

**Compare two versions with real users:**

```python
def assign_model(user_id):
    """Consistent assignment per user"""
    # Hash user ID to ensure same user always gets same version
    if hash(user_id) % 2 == 0:
        return "cora-v1.0"  # Control (A)
    else:
        return "cora-v2.0"  # Treatment (B)

# Track metrics per version
metrics = {
    "cora-v1.0": {"satisfaction": [], "latency": []},
    "cora-v2.0": {"satisfaction": [], "latency": []}
}

# After 1 week, compare
analyze_ab_test(metrics)
# → If v2.0 is better, roll out to everyone
```

**Benefits:**
- Data-driven decisions
- Measure real impact
- Optimize for business metrics (satisfaction, conversion)

---

## Monitoring and Scaling

### Key Metrics to Monitor

**1. Usage Metrics**

```python
# Track over time
daily_queries = count_requests(period="day")
tokens_consumed = sum_tokens(period="day")
unique_users = count_unique_users(period="day")

# Compare to capacity
utilization = tokens_consumed / total_capacity
if utilization > 0.8:
    alert("High utilization - consider scaling")
```

**2. Performance Metrics**

```python
# Response times
avg_latency = mean_response_time()
p95_latency = percentile_95_response_time()
p99_latency = percentile_99_response_time()

# Targets
assert avg_latency < 1500  # 1.5s average
assert p95_latency < 3000  # 3s for 95% of requests
```

**3. Quality Metrics**

```python
# Sample responses daily
daily_sample = random_sample(size=100)

quality_scores = evaluate(
    data=daily_sample,
    evaluators=quality_evaluators
)

if quality_scores.average < threshold:
    alert("Quality degradation detected")
```

**4. Cost Metrics**

```python
# Track spending
daily_cost = tokens_used * price_per_token
monthly_projection = daily_cost * 30

if monthly_projection > budget:
    alert(f"Projected to exceed budget: ${monthly_projection}")
```

### Scaling Strategies

**Vertical Scaling (increase capacity):**

```python
# Current: 10K TPM
# Need: 25K TPM

update_deployment(
    name="cora-prod",
    capacity=25_000  # Scale up
)
```

**Horizontal Scaling (multiple deployments):**

```python
# Create multiple deployments
deployments = [
    "cora-prod-1",  # 50K TPM
    "cora-prod-2",  # 50K TPM
    "cora-prod-3"   # 50K TPM
]
# Total: 150K TPM

# Load balance across them
def route_query(query):
    deployment = random.choice(deployments)
    return query_model(deployment, query)
```

**Auto-scaling (planned):**

```python
# Define scaling rules
scaling_policy = {
    "metric": "utilization",
    "scale_up": {
        "threshold": 0.8,  # 80% utilization
        "action": "increase_capacity_by_50_percent"
    },
    "scale_down": {
        "threshold": 0.3,  # 30% utilization
        "action": "decrease_capacity_by_25_percent"
    }
}
```

### Alerting

**Set up proactive alerts:**

```python
alerts = [
    {
        "name": "High Error Rate",
        "condition": "error_rate > 0.05",
        "severity": "critical",
        "action": "page_on_call"
    },
    {
        "name": "High Latency",
        "condition": "p95_latency > 3000ms",
        "severity": "warning",
        "action": "notify_team"
    },
    {
        "name": "High Cost",
        "condition": "daily_cost > budget * 1.5",
        "severity": "warning",
        "action": "notify_finance"
    },
    {
        "name": "Quality Degradation",
        "condition": "groundedness < 4.0",
        "severity": "warning",
        "action": "investigate"
    }
]
```

---

## Common Deployment Issues

### Issue 1: Deployment Not Found

**Error:**
```
Error: The deployment 'cora-v1' was not found
```

**Causes:**
- Typo in deployment name
- Deployment in different region
- Deployment deleted

**Solutions:**
```python
# List all deployments
deployments = ml_client.online_deployments.list()
for d in deployments:
    print(d.name)

# Verify exact name
deployment_name = "cora-fine-tuned-v1"  # Use exact name
```

### Issue 2: Insufficient Capacity

**Error:**
```
Error: Rate limit exceeded (429)
```

**Cause:** Too many requests for deployment capacity

**Solutions:**

```python
# Option 1: Increase capacity
update_deployment(capacity=50_000)

# Option 2: Implement retry logic
import time

def query_with_retry(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            return client.query(prompt)
        except RateLimitError:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                raise

# Option 3: Load balance across deployments
```

### Issue 3: High Latency

**Symptom:** Responses take > 5 seconds

**Causes:**
- Overloaded deployment
- Complex queries
- Network issues

**Solutions:**

```python
# 1. Check utilization
if utilization > 0.9:
    scale_up_deployment()

# 2. Optimize prompts (reduce tokens)
prompt = shorten_prompt(original_prompt)

# 3. Use faster model for simple queries
if is_simple_query(query):
    model = "gpt-4o-mini"  # Faster
else:
    model = "cora-fine-tuned"  # Better quality
```

### Issue 4: Quality Regression

**Symptom:** Deployed model performs worse than expected

**Causes:**
- Different data distribution in production
- Fine-tuning overfitting
- Configuration issues

**Solutions:**

```python
# 1. Evaluate on production sample
prod_sample = sample_production_queries(size=100)
results = evaluate(data=prod_sample, evaluators=evaluators)

# 2. Compare to development results
if results.score < dev_results.score - 0.5:
    alert("Quality regression detected")
    
# 3. Retrain with production-like data
new_training_data = augment_with_production_examples()
fine_tune_again(data=new_training_data)
```

---

## Terminology Quick Reference

| Term | Simple Definition |
|------|-------------------|
| **Deployment** | Making a model available for production use via API |
| **Deployment Name** | Friendly name to reference deployed model |
| **Model ID** | Unique identifier from fine-tuning job |
| **Capacity (TPM)** | Tokens per minute the deployment can handle |
| **Blue-Green Deployment** | Two identical environments for zero-downtime updates |
| **Canary Deployment** | Gradual rollout to small percentage of users |
| **A/B Testing** | Comparing two versions with real user traffic |
| **Rollback** | Reverting to previous version if issues occur |
| **Vertical Scaling** | Increasing capacity of single deployment |
| **Horizontal Scaling** | Adding more deployments |
| **Utilization** | Percentage of capacity being used |
| **Latency** | Time to generate a response |
| **Production** | Live environment serving real customers |

---

## What's Next?

Now that you understand deployment concepts, you're ready to deploy your fine-tuned model!

### Hands-On Notebook in This Section

- **`60-deployment.ipynb`** - Deploy your fine-tuned model to production
  - List completed fine-tuning jobs
  - Select model for deployment
  - Create deployment with appropriate capacity
  - Verify deployment is working
  - Test deployed model
  - Update application configuration

### Recommended Learning Path

1. **Start here** → Understand concepts (this notebook)
2. **Next** → Deploy model (`60-deployment.ipynb`)
3. **Then** → Monitor production (Azure AI Foundry portal)
4. **Advanced** → Implement canary deployment
5. **Continuous** → Monitor metrics and scale as needed

### After Deployment

**Continue learning:**
- **Monitoring** → Set up Application Insights dashboards
- **Tracing** → Implement OpenTelemetry for observability
- **Optimization** → Fine-tune capacity based on usage
- **Updates** → Deploy new versions using best practices
- **Scaling** → Handle growth in user traffic

---

## Further Reading

For deeper understanding:

- **[Deploy Fine-Tuned Models](https://learn.microsoft.com/azure/ai-services/openai/how-to/fine-tuning?tabs=turbo%2Cpython-new&pivots=programming-language-python#deploy-a-fine-tuned-model)** - Azure OpenAI deployment guide
- **[Azure AI Foundry Deployment](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models-openai)** - Deployment in AI Foundry
- **[Production Best Practices](https://learn.microsoft.com/azure/ai-services/openai/how-to/production-readiness)** - Production readiness checklist
- **[Monitoring and Logging](https://learn.microsoft.com/azure/ai-services/openai/how-to/monitoring)** - Application Insights integration
- **[Cost Management](https://learn.microsoft.com/azure/ai-services/openai/how-to/manage-costs)** - Optimizing deployment costs

---

Ready to deploy your model? Open `60-deployment.ipynb` to get started! 🚀